# Expérience des k-bandits
On suppose qu'on se trouve en face de k machines à sous ayant chacune leur probabilité de gain que le joueur ne connait pas. Quelles machines à sous actionner et dans quel ordre ?

Cet exemple est un problème de base en apprentissage par renforcement. Voici quelques exemples d'application
* La société de recouvrement doit choisir de charger son dialer avec k numéros de téléphone.

Pour faire l'expérience prenons les hypothèses suivantes :
* Jouer sur un bandit coûte 1
* Il y a k bandits
* Chaque bandit a une probabilité $p$ de faire gagner le joueur
* La distribution de la probabilité de gain suit une loi uniforme sur $\lbrack 0.0.5,0.9 \rbrack$ 
* Le montant gagné suit une loi gamma d'espérance $\mu$ et d'écart type $\sigma$ faite pour qu'en moyenne le joueur perde. Par contre les gains du bandit 0 suivent une loi qui permet en moyenne de gagner

L'objectif est donc de faire découvrir à l'agent qu'il doit jouer le bandit 0

## Caractéristique des bandits

In [1]:
# librairies
import numpy as np
import pandas as pd
import plotly
import plotly.io as pio
import random
# graines
#np.random.seed(1)
#random.seed(1)
# définition de la palette de couleur de Blitz
palette = ['#006699','#70AB35','#F1A022','#DEE23F','#C22532','#D5C8B3']
# définition du layout par défaut de Blitz
layout = {
    'colorway' : palette,
    'font' : { 'color' : palette[0], 'family' :"'Quicksand','Open Sans','Helvetica Neue',Arial,sans-serif"},
    'paper_bgcolor' : 'rgba(0,0,0,0)',
    'plot_bgcolor' : 'rgba(0,0,0,0)',
    'xaxis' : {
        'gridcolor' : palette[5],
        'linecolor' : palette[0],
        'color' : palette[0]
    },
    'yaxis' : {
        'gridcolor' : palette[5],
        'linecolor' : palette[0],
        'color' : palette[0]
    },
    'margin' : {'t' : 30}
} 
fig = {'data':[],'layout':layout}
# Sauvegarde du template
templated_fig = pio.to_templated(fig)
pio.templates['blitz'] = templated_fig.layout.template

On choisit de jouer avec k bandits. Chacun a un probabilité que le joueur gagne qui suit une loi uniforme sur $\lbrack 0.05,0.9 \rbrack$

In [2]:
k = 8
minP = 0.05
maxP = 0.9

On choisit les probabilité que le joueur réussisse pour chaque bandit

In [3]:
from scipy.stats import uniform
bandits = pd.DataFrame({
    'p':uniform.rvs(loc=minP, scale=maxP-minP, size=k, random_state=None),
})
bandits

,p
0,0.428904
1,0.138952
2,0.827658
3,0.385794
4,0.644001
5,0.254250
6,0.412105
7,0.330795


Le montant des gains suit une loi gamma d'espérance $\mu = \frac{0.80}{p}$ sauf le bandit 0 qui a une espérance de  $\mu = \frac{1.05}{p}$ et d'écart type $\sigma = \frac{\mu}{1.02}$

Le formalisme de scipy décrit la loi gamma avec un facteur de forme `a` et un facteur d'échelle `scale`. On montre (voir [loi gamma](https://fr.wikipedia.org/wiki/Loi_Gamma)) que `a`$=\frac{\mu^2}{\sigma^2}$ et `scale`=$\frac{\sigma^2}{\mu}$.

In [4]:
bandits['mu'] = 0.80/bandits.p
bandits.loc[0,'mu'] = 1.05/bandits.p[0]
bandits['sigma'] = bandits.mu/1.02
bandits['a'] =bandits.mu**2/bandits.sigma**2
bandits['scale'] = bandits.sigma**2/bandits.mu
bandits

,p,mu,sigma,a,scale
0,0.428904,2.448101,2.400099,1.0404,2.353038
1,0.138952,5.757395,5.644505,1.0404,5.533829
2,0.827658,0.966583,0.947631,1.0404,0.929050
3,0.385794,2.073647,2.032987,1.0404,1.993124
4,0.644001,1.242234,1.217877,1.0404,1.193997
5,0.254250,3.146508,3.084812,1.0404,3.024325
6,0.412105,1.941252,1.903188,1.0404,1.865871
7,0.330795,2.418415,2.370996,1.0404,2.324505


On représente la densité de probabilité de gain en cas de succès pour chacun des bandits

In [5]:
from scipy.stats import gamma
reward = np.arange(0,5,0.001)
densite = pd.DataFrame({'bandit':[],'reward':[],'densite':[]})
for i in bandits.index:
    d = gamma.pdf(reward,a=bandits.a[i],loc=0,scale=bandits.scale[i])
    banditId = np.repeat(i,reward.shape[0])
    df = pd.DataFrame({'bandit':banditId,'reward':reward,'densite':d})
    densite = pd.concat([densite, df])

In [6]:
data_fig=[]

for i in bandits.index:
    data_fig.append({
    'type': 'scatter',
    'mode' : 'lines',
    'name' : 'bandit ' + str(i),
    'x' : densite[densite.bandit == i].reward,
    'y' : densite[densite.bandit == i].densite 
    })

layout = {  
    'title' : 'distribution de gain des bandits',
    'template' : 'blitz',
    'xaxis' : {
        'title' : "gain",
    },
    'yaxis' : {
        'title' : "densité de probabilité",
        'tickformat' : '.2f'
    }
} 
pio.show({'data': data_fig, 'layout':layout})

## Stratégie de choix aléatoire des bandits
On choisit les bandits avec une probabilité uniforme de $\frac{1}{k}$. On observe les gains suite à 3000 tirages. On note `R` le gain de chaque tirage et `Q` le gain moyen au cours des tirages.

In [7]:
from scipy.stats import bernoulli
nTirages = 3000
Q = 0
tirageAleatoire = pd.DataFrame({'bandit':[],'win':[],'R':[],'Q':[]})
for i in np.arange(0,nTirages,1):
    # le joueur choisir un bandit
    bandit = random.choices(bandits.index)
    # le joueur met 1 € dans la machine
    R = [-1]
    # le joueur lance la machine
    win = bernoulli.rvs(bandits.p[bandit], loc=0, size=1, random_state=None)
    # si il a gagné il reçoit son gain
    gain = win*gamma.rvs(a = bandits.a[bandit], loc=0, scale=bandits.scale[bandit], size=1, random_state=None)
    # ce gain se cumule avec ce qu'il avait mis dans la machine
    R = R + gain
    # on peut calculer le nouveau gain moyen
    Q = 1/(i+1)*R + i/(i+1)*Q
    # on ajoute tout cela dans le tableau
    T = pd.DataFrame({'bandit':bandit,'win':win,'R':R,'Q':Q}, index=[i])
    tirageAleatoire = pd.concat([tirageAleatoire,T])

On peut observer l'évolution des gains

In [8]:
data_fig=[{
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageAleatoire.index,
    'y' : tirageAleatoire.Q
}]


layout = {  
    'title' : 'évolution des gains avec un tirage aléatoire des bandits',
    'template' : 'blitz',
    'xaxis' : {
        'title' : "tirage",
    },
    'yaxis' : {
        'title' : "gains moyens",
        'tickformat' : '.2f',
        'ticksuffix' : ' €'
    }
} 
pio.show({'data': data_fig, 'layout':layout})

Total des pertes du joueur

In [9]:
tirageAleatoire.R.sum()

-350.636797690579

# Stratégie epsilon greedy

La stratégie epsilon greedy consiste à choisir aléatoirement avec une probabilité $\epsilon$ entre l'**exploration** (on teste une autre machine que la meilleure) et l'**exploitation** (on utilise la machine qui rend en moyenne le meilleur résultat)

On choisit $\epsilon = 0.1$ c'est à dire qu'en moyenne dans un 1 cas sur 10, on choisira d'explorer plutôt que de d'exploiter

On commence par créer la fonction `choisirBanditEg` qui permet de choisir un bandit par la méthode epsilon greedy

On va avoir besoin pour cela de connaitre la récompense moyenne courante pour chacun des bandits et ajouter l'information sur le nombre de fois où le bandit a été utilisé

In [10]:
bandits['Q'] = 0
bandits['n'] = 0
bandits

,p,mu,sigma,a,scale,Q,n
0,0.428904,2.448101,2.400099,1.0404,2.353038,0,0
1,0.138952,5.757395,5.644505,1.0404,5.533829,0,0
2,0.827658,0.966583,0.947631,1.0404,0.929050,0,0
3,0.385794,2.073647,2.032987,1.0404,1.993124,0,0
4,0.644001,1.242234,1.217877,1.0404,1.193997,0,0
5,0.254250,3.146508,3.084812,1.0404,3.024325,0,0
6,0.412105,1.941252,1.903188,1.0404,1.865871,0,0
7,0.330795,2.418415,2.370996,1.0404,2.324505,0,0


Nous allons maintenant coder la fonction `choisirBanditEg`

In [11]:
epsilon = 0.1
def choisirBanditEg():
    # on cherche la récompense moyenne la plus élevée
    maxQ = bandits.Q.max()
    # on tire au hasard si on exploite ou on explore
    e = bernoulli.rvs(epsilon, loc=0, size=1, random_state=None)[0]
    # on choisit d'exploiter
    if (e==0):
        # on tire au hasard parmi les bandits qui ont la plus grande valeur d'action
        return random.choices(bandits[bandits.Q == maxQ].index)[0]
    # sinon on choisit d'explorer
    else:
        # on tire au hasard parmi les bandits qui n'ont pas la plus grande valeur d'action
        return random.choices(bandits[bandits.Q != maxQ].index)[0]        

Maintenant on peut utiliser cette méthode dans l'algorithme pour les tirages

In [12]:
Q = bandits.Q.max()
tirageEQ = pd.DataFrame({'bandit':[],'win':[],'R':[],'Q':[]})
for i in np.arange(0,nTirages,1):
    # le joueur choisir un bandit
    bandit = choisirBanditEg()
    # le joueur met 1 € dans la machine
    R = [-1]
    # le joueur lance la machine
    win = bernoulli.rvs(bandits.p[bandit], loc=0, size=1, random_state=None)
    # si il a gagné il reçoit son gain
    gain = win*gamma.rvs(a = bandits.a[bandit], loc=0, scale=bandits.scale[bandit], size=1, random_state=None)
    # ce gain se cumule avec ce qu'il avait mis dans la machine
    R = R + gain
    # on peut calculer le nouveau gain moyen
    Q = 1/(i+1)*R + i/(i+1)*Q
    # on ajoute tout cela dans le tableau
    T = pd.DataFrame({'bandit':bandit,'win':win,'R':R,'Q':Q}, index=[i])
    # on met à jour le nouvelle valeur d'action du bandit utilisé
    oldQ = bandits.loc[bandit,'Q']
    n = bandits.loc[bandit,'n']
    newQ = 1/(n+1)*R + n/(n+1)*oldQ
    bandits.loc[bandit,'n'] = n+1
    bandits.loc[bandit,'Q'] = newQ
    # on concatène les résultats pour suivre les tirages
    tirageEQ = pd.concat([tirageEQ,T])

In [13]:
bandits

,p,mu,sigma,a,scale,Q,n
0,0.428904,2.448101,2.400099,1.0404,2.353038,0.088569,2500
1,0.138952,5.757395,5.644505,1.0404,5.533829,-0.179349,186
2,0.827658,0.966583,0.947631,1.0404,0.929050,-0.266595,43
3,0.385794,2.073647,2.032987,1.0404,1.993124,-0.280668,60
4,0.644001,1.242234,1.217877,1.0404,1.193997,-0.355888,48
5,0.254250,3.146508,3.084812,1.0404,3.024325,-0.430177,48
6,0.412105,1.941252,1.903188,1.0404,1.865871,-0.343336,51
7,0.330795,2.418415,2.370996,1.0404,2.324505,-0.181261,64


Si on attend assez longtemps, on voit que c'est bien le bandit 0 qui a la valeur d'action la plus élevée et qui fini par être choisi.

In [14]:
data_fig=[{
    'name' : "hasard",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageAleatoire.index,
    'y' : tirageAleatoire.Q
},{
    'name' : "EG",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageEQ.index,
    'y' : tirageEQ.Q
}]

layout = {  
    'title' : 'évolution des gains avec un choix epsilon greedy',
    'template' : 'blitz',
    'xaxis' : {
        'title' : "tirage",
    },
    'yaxis' : {
        'title' : "valeur d'action",
        'tickformat' : '.2f',
        'ticksuffix' : ' €'
    }
} 
pio.show({'data': data_fig, 'layout':layout})

## Optimistic initial value
Le principe est de partir d'une valeur d'action optimiste pour chacune des actions afin de forcer l'exploration au début. Les valeurs vont baisser au fur et à mesure des résultats. Dès qu'une valeur va avoir trop baisser c'est une autre qui va être explorée.

Il faut donc calculer la valeur d'action moyenne réelle et la valeur d'action moyenne optimiste pour chaque bandit

In [15]:
bandits['Qoptimistic'] = 10
bandits['Q'] = 0
bandits['n'] = 0
bandits

,p,mu,sigma,a,scale,Q,n,Qoptimistic
0,0.428904,2.448101,2.400099,1.0404,2.353038,0,0,10
1,0.138952,5.757395,5.644505,1.0404,5.533829,0,0,10
2,0.827658,0.966583,0.947631,1.0404,0.929050,0,0,10
3,0.385794,2.073647,2.032987,1.0404,1.993124,0,0,10
4,0.644001,1.242234,1.217877,1.0404,1.193997,0,0,10
5,0.254250,3.146508,3.084812,1.0404,3.024325,0,0,10
6,0.412105,1.941252,1.903188,1.0404,1.865871,0,0,10
7,0.330795,2.418415,2.370996,1.0404,2.324505,0,0,10


On définit un nouvelle fonction de choix

In [16]:
def choisirBanditOpt():
    # on cherche la récompense moyenne optmisite la plus élevée
    maxQ = bandits.Qoptimistic.max()
    # on tire au hasard parmi les bandits qui ont la plus grande valeur d'action
    return random.choices(bandits[bandits.Qoptimistic == maxQ].index)[0]

On exploite ce nouvelle fonction dans l'algorithme de choix

In [17]:
Q = 0
Qoptimistic = bandits.Qoptimistic.max()
tirageOpt = pd.DataFrame({'bandit':[],'win':[],'R':[],'Q':[]})
for i in np.arange(0,nTirages,1):
    # le joueur choisir un bandit
    bandit = choisirBanditOpt()
    # le joueur met 1 € dans la machine
    R = [-1]
    # le joueur lance la machine
    win = bernoulli.rvs(bandits.p[bandit], loc=0, size=1, random_state=None)
    # si il a gagné il reçoit son gain
    gain = win*gamma.rvs(a = bandits.a[bandit], loc=0, scale=bandits.scale[bandit], size=1, random_state=None)
    # ce gain se cumule avec ce qu'il avait mis dans la machine
    R = R + gain
    # on peut calculer le nouveau gain moyen
    Q = 1/(i+1)*R + i/(i+1)*Q
    # on peut claculer le nouveau gain optimiste
    Qoptimistic = 1/(i+2)*R + (i+1)/(i+2)*Qoptimistic
    # on ajoute tout cela dans le tableau
    T = pd.DataFrame({'bandit':bandit,'win':win,'R':R,'Q':Q}, index=[i])
    # on met à jour le nouvelle valeur d'action du bandit utilisé
    oldQ = bandits.loc[bandit,'Q']
    n = bandits.loc[bandit,'n']
    newQ = 1/(n+1)*R + n/(n+1)*oldQ
    bandits.loc[bandit,'n'] = n+1
    bandits.loc[bandit,'Q'] = newQ
    # on met à jour le nouvelle valeur d'action optimiste du bandit utilisé  
    # attentionn au début il faut faire la moyenne entre le Qoptimistic et R
    oldQoptimistic = bandits.loc[bandit,'Qoptimistic']
    newQoptimistic = 1/(n+2)*R + (n+1)/(n+2)*oldQoptimistic
    bandits.loc[bandit,'Qoptimistic'] = newQoptimistic    
    # on concatène les résultats pour suivre les tirages
    tirageOpt = pd.concat([tirageOpt,T])

Le système arrive bien à trouver que le bandit 0 est le meilleur

In [18]:
bandits

,p,mu,sigma,a,scale,Q,n,Qoptimistic
0,0.428904,2.448101,2.400099,1.0404,2.353038,0.101960,2541,0.105854
1,0.138952,5.757395,5.644505,1.0404,5.533829,-0.208786,48,-0.000443
2,0.827658,0.966583,0.947631,1.0404,0.929050,-0.377179,27,-0.006565
3,0.385794,2.073647,2.032987,1.0404,1.993124,-0.031841,209,0.015930
4,0.644001,1.242234,1.217877,1.0404,1.193997,-0.363551,27,0.006576
5,0.254250,3.146508,3.084812,1.0404,3.024325,-0.068151,119,0.015750
6,0.412105,1.941252,1.903188,1.0404,1.865871,-0.542637,18,0.012239
7,0.330795,2.418415,2.370996,1.0404,2.324505,-0.958114,11,-0.044938


In [19]:
data_fig=[{
    'name' : "hasard",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageAleatoire.index,
    'y' : tirageAleatoire.Q
},{
    'name' : "EQ",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageEQ.index,
    'y' : tirageEQ.Q
},{
    'name' : "Optimistic",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageOpt.index,
    'y' : tirageOpt.Q
}]

layout = {  
    'title' : 'évolution des gains avec un choix optimiste',
    'template' : 'blitz',
    'xaxis' : {
        'title' : "tirage",
    },
    'yaxis' : {
        'title' : "valeur d'action",
        'tickformat' : '.2f',
        'ticksuffix' : ' €'
    }
} 
pio.show({'data': data_fig, 'layout':layout})

In [20]:
tirageOpt

,bandit,win,R,Q
0,6.0,0.0,-1.000000,-1.000000
1,7.0,0.0,-1.000000,-1.000000
2,4.0,1.0,-0.988870,-0.996290
3,0.0,1.0,1.221317,-0.441888
4,2.0,1.0,-0.539242,-0.461359
...,...,...,...,...
2995,0.0,0.0,-1.000000,0.065945
2996,0.0,0.0,-1.000000,0.065589
2997,0.0,0.0,-1.000000,0.065234
2998,0.0,1.0,-0.583182,0.065017


## Upper Confidence Bound (UCB)
L'idée des UCB est d'ajouter une incertitude à l'estimation que fait $Q(a)$ de $q_*(a)$. Plus on teste l'option $a$, plus l'incertitude diminue. On démarre ainsi au hasard, puis on choisit l'UCB le plus élevé.

La formule utilisée pour l'UCB est
$$ UCB_t(a) = Q_t(a) + c \sqrt{\frac{ln(N_{tirages})}{N_t(a)}}$$
où $N_{tirages}N_{tirages}$ est le nombre de tirage et $N_t(a)$ est le nombre de fois où $a$ a été choisi

On va commencer par créer la variable `UCB` qui va nous servir

In [21]:
bandits['UCB'] = 0
bandits['Q'] = 0
bandits['n'] = 0
bandits['Qoptimistic'] = 0
bandits

,p,mu,sigma,a,scale,Q,n,Qoptimistic,UCB
0,0.428904,2.448101,2.400099,1.0404,2.353038,0,0,0,0
1,0.138952,5.757395,5.644505,1.0404,5.533829,0,0,0,0
2,0.827658,0.966583,0.947631,1.0404,0.929050,0,0,0,0
3,0.385794,2.073647,2.032987,1.0404,1.993124,0,0,0,0
4,0.644001,1.242234,1.217877,1.0404,1.193997,0,0,0,0
5,0.254250,3.146508,3.084812,1.0404,3.024325,0,0,0,0
6,0.412105,1.941252,1.903188,1.0404,1.865871,0,0,0,0
7,0.330795,2.418415,2.370996,1.0404,2.324505,0,0,0,0


On définit une nouvelle variable de choix

In [22]:
def choisirBanditUCB():
    # on cherche la récompense moyenne optmisite la plus élevée
    maxUCB = bandits.UCB.max()
    # on tire au hasard parmi les bandits qui ont la plus grande UCB
    return random.choices(bandits[bandits.UCB == maxUCB].index)[0]

On peut alors faire une nouvelle série de tirage. Avant il faut choisir un coefficient $c$ et initialiser le système

In [23]:
import math
Q = 0
UCB = bandits.UCB.max()
c = 1
tirageUCB = pd.DataFrame({'bandit':[],'win':[],'R':[],'Q':[]})

On fait un premier tour en tirant une fois tous les bandits

In [24]:
for i in np.arange(0,k,1):
    # le joueur choisit un bandit
    bandit = choisirBanditUCB()
    # le joueur met 1 € dans la machine
    R = [-1]
    # le joueur lance la machine
    win = bernoulli.rvs(bandits.p[i], loc=0, size=1, random_state=None)
    # si il a gagné il reçoit son gain
    gain = win*gamma.rvs(a = bandits.a[i], loc=0, scale=bandits.scale[i], size=1, random_state=None)
    # ce gain se cumule avec ce qu'il avait mis dans la machine
    R = R + gain
    # on peut calculer le nouveau gain moyen
    Q = 1/(i+1)*R + i/(i+1)*Q
    # on ajoute tout cela dans le tableau
    T = pd.DataFrame({'bandit':i,'win':win,'R':R,'Q':Q}, index=[i])
    # on met à jour le nouvelle valeur d'action du bandit utilisé
    oldQ = bandits.loc[i,'Q']
    n = bandits.loc[i,'n']
    newQ = 1/(n+1)*R + n/(n+1)*oldQ
    bandits.loc[i,'n'] = n+1
    bandits.loc[i,'Q'] = newQ
    # on met à jour l'UCB du bandit en se positionnant d'emblée à la fin de ce primier tour
    bandits.loc[i,'UCB'] = bandits.Q[i] + c*math.sqrt(math.log(k)/(bandits.n[i]))    
    # on concatène les résultats pour suivre les tirages
    tirageUCB = pd.concat([tirageUCB,T])

On vérifie les tirages

In [25]:
tirageUCB

,bandit,win,R,Q
0,0.0,1.0,3.734189,3.734189
1,1.0,0.0,-1.000000,1.367094
2,2.0,1.0,-0.868789,0.621800
3,3.0,0.0,-1.000000,0.216350
4,4.0,1.0,-0.511786,0.070723
5,5.0,0.0,-1.000000,-0.107731
6,6.0,1.0,1.971345,0.189280
7,7.0,0.0,-1.000000,0.040620


On vérifie l'état des bandits

In [26]:
bandits

,p,mu,sigma,a,scale,Q,n,Qoptimistic,UCB
0,0.428904,2.448101,2.400099,1.0404,2.353038,3.734189,1,0,5.176215
1,0.138952,5.757395,5.644505,1.0404,5.533829,-1.000000,1,0,0.442027
2,0.827658,0.966583,0.947631,1.0404,0.929050,-0.868789,1,0,0.573238
3,0.385794,2.073647,2.032987,1.0404,1.993124,-1.000000,1,0,0.442027
4,0.644001,1.242234,1.217877,1.0404,1.193997,-0.511786,1,0,0.930241
5,0.254250,3.146508,3.084812,1.0404,3.024325,-1.000000,1,0,0.442027
6,0.412105,1.941252,1.903188,1.0404,1.865871,1.971345,1,0,3.413372
7,0.330795,2.418415,2.370996,1.0404,2.324505,-1.000000,1,0,0.442027


On lance les tirages pour la suite

In [27]:
for i in np.arange(k+1,nTirages,1):
    # le joueur choisit un bandit
    bandit = choisirBanditUCB()
    # le joueur met 1 € dans la machine
    R = [-1]
    # le joueur lance la machine
    win = bernoulli.rvs(bandits.p[bandit], loc=0, size=1, random_state=None)
    # si il a gagné il reçoit son gain
    gain = win*gamma.rvs(a = bandits.a[bandit], loc=0, scale=bandits.scale[bandit], size=1, random_state=None)
    # ce gain se cumule avec ce qu'il avait mis dans la machine
    R = R + gain
    # on peut calculer le nouveau gain moyen
    Q = 1/(i+1)*R + i/(i+1)*Q
    # on ajoute tout cela dans le tableau
    T = pd.DataFrame({'bandit':bandit,'win':win,'R':R,'Q':Q}, index=[i])
    # on met à jour le nouvelle valeur d'action du bandit utilisé
    oldQ = bandits.loc[bandit,'Q']
    n = bandits.loc[bandit,'n']
    newQ = 1/(n+1)*R + n/(n+1)*oldQ
    bandits.loc[bandit,'n'] = n+1
    bandits.loc[bandit,'Q'] = newQ
    # on met à jour les nouvelles UCB pour tous les bandits
    for b in np.arange(0,k,1):
        bandits.loc[b,'UCB'] = bandits.Q[b] + c*math.sqrt(math.log(i+1)/(bandits.n[b]))    
    # on concatène les résultats pour suivre les tirages
    tirageUCB = pd.concat([tirageUCB,T])

On vérifie que l'agent arrive à trouver le bon bandit

In [28]:
bandits

,p,mu,sigma,a,scale,Q,n,Qoptimistic,UCB
0,0.428904,2.448101,2.400099,1.0404,2.353038,0.042597,2593,0,0.098164
1,0.138952,5.757395,5.644505,1.0404,5.533829,-0.191846,111,0,0.076723
2,0.827658,0.966583,0.947631,1.0404,0.929050,-0.351638,43,0,0.079865
3,0.385794,2.073647,2.032987,1.0404,1.993124,-1.000000,7,0,0.069470
4,0.644001,1.242234,1.217877,1.0404,1.193997,-0.193406,108,0,0.078868
5,0.254250,3.146508,3.084812,1.0404,3.024325,-0.199673,101,0,0.081878
6,0.412105,1.941252,1.903188,1.0404,1.865871,-0.598184,18,0,0.068748
7,0.330795,2.418415,2.370996,1.0404,2.324505,-0.591651,18,0,0.075281


In [29]:
data_fig=[{
    'name' : "hasard",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageAleatoire.index,
    'y' : tirageAleatoire.Q
},{
    'name' : "EQ",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageEQ.index,
    'y' : tirageEQ.Q
},{
    'name' : "Optimistic",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageOpt.index,
    'y' : tirageOpt.Q
},{
    'name' : "UCB",
    'type': 'scatter',
    'mode' : 'lines',
    'x' : tirageUCB.index,
    'y' : tirageUCB.Q
}]

layout = {  
    'title' : 'évolution des gains avec un choix UCB',
    'template' : 'blitz',
    'xaxis' : {
        'title' : "tirage",
    },
    'yaxis' : {
        'title' : "valeur d'action",
        'tickformat' : '.2f',
        'ticksuffix' : ' €'
    }
} 
pio.show({'data': data_fig, 'layout':layout})